In [1]:
import arcade

# Constants
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 400
PADDLE_WIDTH = 100
PADDLE_HEIGHT = 10
BALL_RADIUS = 10
BRICK_WIDTH = 58
BRICK_HEIGHT = 20
BRICK_ROWS = 5
BRICK_COLS = 10
BRICK_PADDING = 2

class DXBallGame(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, "DX Ball Game")
        arcade.set_background_color(arcade.color.BLACK)

        # Paddle
        self.paddle = arcade.SpriteSolidColor(PADDLE_WIDTH, PADDLE_HEIGHT, arcade.color.BLUE)
        self.paddle.center_x = SCREEN_WIDTH // 2
        self.paddle.center_y = 50

        # Ball
        self.ball = arcade.SpriteSolidColor(BALL_RADIUS * 2, BALL_RADIUS * 2, arcade.color.RED)
        self.ball.center_x = SCREEN_WIDTH // 2
        self.ball.center_y = SCREEN_HEIGHT // 2
        self.ball.change_x = 3
        self.ball.change_y = 3

        # Bricks
        self.bricks = arcade.SpriteList()
        self.create_bricks()

        self.game_over = False

    def create_bricks(self):
        for row in range(BRICK_ROWS):
            for col in range(BRICK_COLS):
                x = BRICK_WIDTH * col + BRICK_WIDTH // 2 + BRICK_PADDING * col
                y = SCREEN_HEIGHT - (BRICK_HEIGHT * row + BRICK_HEIGHT // 2 + BRICK_PADDING * row)
                brick = arcade.SpriteSolidColor(BRICK_WIDTH, BRICK_HEIGHT, arcade.color.GREEN)
                brick.center_x = x
                brick.center_y = y
                self.bricks.append(brick)

    def on_draw(self):
        arcade.start_render()
        self.paddle.draw()
        self.ball.draw()
        self.bricks.draw()

        if self.game_over:
            arcade.draw_text("GAME OVER", SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2, arcade.color.WHITE, 30, anchor_x="center")
            arcade.draw_text("Press R to Restart or Q to Quit", SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2 - 40, arcade.color.WHITE, 20, anchor_x="center")

    def on_update(self, delta_time):
        if self.game_over:
            return

        # Move ball
        self.ball.center_x += self.ball.change_x
        self.ball.center_y += self.ball.change_y

        # Ball collision with walls
        if self.ball.left <= 0 or self.ball.right >= SCREEN_WIDTH:
            self.ball.change_x *= -1
        if self.ball.top >= SCREEN_HEIGHT:
            self.ball.change_y *= -1

        # Ball collision with paddle
        if (self.paddle.center_x - PADDLE_WIDTH / 2 <= self.ball.center_x <= self.paddle.center_x + PADDLE_WIDTH / 2
                and self.paddle.center_y + PADDLE_HEIGHT / 2 >= self.ball.center_y - BALL_RADIUS):
            self.ball.change_y *= -1

        # Ball collision with bricks
        brick_hit_list = arcade.check_for_collision_with_list(self.ball, self.bricks)
        if brick_hit_list:
            self.ball.change_y *= -1
            for brick in brick_hit_list:
                brick.remove_from_sprite_lists()

        # Check if the ball falls off the screen (Game Over)
        if self.ball.bottom < 0:
            self.game_over = True

        # Move paddle based on key state
        if self.paddle.change_x != 0:
            self.paddle.center_x += self.paddle.change_x
            if self.paddle.left < 0:
                self.paddle.left = 0
            elif self.paddle.right > SCREEN_WIDTH:
                self.paddle.right = SCREEN_WIDTH

    def on_key_press(self, key, modifiers):
        if self.game_over:
            if key == arcade.key.R:  # Restart the game
                self.reset_game()
            elif key == arcade.key.Q:  # Quit the game
                arcade.close_window()
        else:
            if key == arcade.key.LEFT:
                self.paddle.change_x = -5
            elif key == arcade.key.RIGHT:
                self.paddle.change_x = 5

    def on_key_release(self, key, modifiers):
        if not self.game_over:
            self.paddle.change_x = 0

    def reset_game(self):
        self.game_over = False
        self.ball.center_x = SCREEN_WIDTH // 2
        self.ball.center_y = SCREEN_HEIGHT // 2
        self.ball.change_x = 3
        self.ball.change_y = 3
        self.bricks = arcade.SpriteList()
        self.create_bricks()
        self.paddle.center_x = SCREEN_WIDTH // 2

def main():
    game = DXBallGame()
    arcade.run()

if __name__ == "__main__":
    main()
